In [ ]:
# https://www.kdnuggets.com/2018/11/multi-class-text-classification-model-comparison-selection.html/2

In [ ]:
# Code to perform classification using classical ML models
# Naive Bayes
# SGDClassifier
# Logistic Regression


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

print('done')

done


In [ ]:
import pandas as pd
import numpy as np

df_2 = pd.read_csv("./2_man_ann_sb.csv",  index_col= None)
df_2 = df_2.dropna(subset = ['tweet_text'])

df_1 = pd.read_csv("./mann_ann_sb.csv", index_col= None)

df_3 = pd.read_csv("./3_man_ann_sb_full_1.csv", index_col = None)
df_3 = df_3.dropna(subset = ['tweet_text'])

df_raw = df_1.append(df_2).append(df_3) # using batch 1 and batch 2 for training
df = df_raw
df_raw.shape

(7500, 19)

Data Cleaning

In [ ]:
# Remove ads marked as below because they are not available in ad annotations file although tweets mention them
# commercials, joe biden, pizzahut, joe bieden, michael bloomberg, mike bloomberg, scientology...
ads_remove = ['commercials', 'joe biden', 'pizzahut', 'joe bieden', 'michael bloomberg', 'mike bloomberg', 
              'scientology','papa johns',  'bakari',  'secret', 'dashlane', 'bernie the peoples perfume',
              'ram trucks', 'golden gronks', "bush's best", 'ragged old flag', 'patience', 'guitar hero',
              'disney mulan']

# ads_rename = ['nfl100', 'tide']

# rename ads with spelling faults while manually adding the annotation 

df_raw['ad_manual_adjusted'] = df_raw['ad_manual_adjusted'].apply(lambda x: x.lower())
df_raw.loc[df_raw.ad_manual_adjusted == "discover card  no we don‚äôt charge annual fees", 
       "ad_manual_adjusted"] = "discover card  no we don’t charge annual fees"
df_raw.loc[df_raw.ad_manual_adjusted == "doritos the cool ranch", 
       "ad_manual_adjusted"] = "doritos  the cool ranch"
df_raw.loc[df_raw.ad_manual_adjusted == "discover card yes we're accepted", 
       "ad_manual_adjusted"] =  "discover card  yes we’re accepted"
df_raw.loc[df_raw.ad_manual_adjusted == "discover card yes we’re accepted", 
       "ad_manual_adjusted"] =  "discover card  yes we’re accepted"
df_raw.loc[df_raw.ad_manual_adjusted == "discover card  yes we're accepted", 
       "ad_manual_adjusted"] =  "discover card  yes we’re accepted"
df_raw.loc[df_raw.ad_manual_adjusted == "budweiser typical american", 
       "ad_manual_adjusted"] = "budweiser  typical american"
df_raw.loc[df_raw.ad_manual_adjusted == 'fox  halftime show  teaser_3',
            "ad_manual_adjusted"] = "fox  halftime show  teaser_1"
df_raw.loc[df_raw.ad_manual_adjusted == 'fox  halftime show  teaser_2',
            "ad_manual_adjusted"] = "fox  halftime show  teaser_1"
            
print("df_raw: " + str(df_raw.shape))
df = pd.DataFrame()
removed_Data = pd.DataFrame()

for i,row  in df_raw.iterrows():
  if row['ad_manual_adjusted'] not in ads_remove:
    df = df.append(row)
  else:
    removed_Data = removed_Data.append(row)

print("df: " + str(df.shape))


df_raw: (7500, 19)
df: (7394, 19)


In [ ]:
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')

# stop = stopwords.words('english')
stop = []
stop.append('superbowl')
stop.append('super')
stop.append('bowl')
stop.append('commercial')
stop.append('ad')
stop.append('commercials')
stop.append('ads')

print(len(stop))

def removeMentions(text):

    textBeforeMention = text.partition("@")[0]
    textAfterMention = text.partition("@")[2]
    textAfterMention =  re.sub(r':', '', textAfterMention) #cadillac join the 31k
    tHandle = textAfterMention.partition(" ")[0].lower() #cadillac    
    text = textBeforeMention+ " " + textAfterMention  
    return text

def cleanTweet(strinp):
    strinp = re.sub(r'RT', "", strinp) # Remove RT
    strinp = strinp.lower()
    
    stop_removed_list = [word for word in strinp.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_list])    
    text = re.sub('https?://[A-Za-z0-9./]+', ' ', stop_removed) # Remove URLs
    text = removeMentions(text)
    text = re.sub('[^\x00-\x7F]+', ' ', text) # Remove non-ASCII chars.
    
    # remove punctuations except '-'
    punctuation = ['(', ')', '[',']','?', ':', ':', ',', '.', '!', '/', '"', "'", '@', '#', '&', '-', '_']
    text = "".join((char for char in text if char not in punctuation))
    text = re.sub('[^a-zA-Z]', ' ', text) # remove all other than alphabet chars 

#     text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters     
    stop_removed_l = [word for word in text.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_l]) 
    return stop_removed

print(cleanTweet("RT @cadillacabc: Joinrt the 31K james_bond") )

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
7
cadillacabc joinrt the k jamesbond


In [ ]:
df['text_clean'] = df['tweet_text'].apply(lambda x: cleanTweet(x))
df['ad_manual_adjusted'] = df['ad_manual_adjusted'].apply(lambda x: x.lower())
df['ad_related'] = df['ad_manual_adjusted'].apply(lambda ad: 0 if ad == 'none' else 1)

comma_filter = ~df['ad_manual_adjusted'].str.contains(',')
df = df[comma_filter]

In [ ]:
df_unique = df.drop_duplicates(subset = ['text_clean'])
df_with_dupes = df
df = df_unique

print(df_with_dupes.shape)
print(df.shape)

(7332, 21)
(5845, 21)


In [ ]:
ad_product_df = pd.read_csv('./SB_ad_annotations_product_category_modified.csv')
ad_product_df = ad_product_df.rename(columns = {'Ad Name': 'Ad_Name'}) # rename the column to remove space
ad_product_df = ad_product_df.dropna() # because the file has trailing empty rows, remove them
# remove fox half time show teaser_2 ad because its keywords is same as fox half time show teaser_2
print(ad_product_df.shape)
ad_product_df.drop(ad_product_df[ad_product_df['Ad_Name'] == 'FOX  Halftime Show  Teaser_2'].index, inplace = True) 
print(ad_product_df.shape)
ad_product_dict = dict()

for i, row in ad_product_df.iterrows():
  ad_product_dict[row['Ad_Name'].lower()] = row['Product_modified'].lower()

ad_product_dict['none'] = 'none'

print(ad_product_dict)

df['product_modified'] = df['ad_manual_adjusted'].apply(lambda ad: ad_product_dict[ad])

(75, 11)
(74, 11)
{'fast & furious 9  trailer': 'movie trailer', 'quibi  bank heist': 'tech company', 'tide  when is later  masked singer': 'laundry detergent', 'fox  a run at history  daytona 500': 'sporting event', 'donald j. trump for president  criminal justice reform': 'political campaign', 'walmart  famous visitors': 'tech company', 'marvel  black widow trailer': 'movie trailer', 'rocket mortgage  home': 'money', 'porsche  the heist': 'car', 'snickers  fix the world': 'food', 'hulu  tom brady s big announcement': 'streaming service', 'fox  chosen  lego masters': 'tv show/network', 'mountain dew': 'pop/soda', 'squarespace  winona in winona': 'tech company', 'new york life  love takes action': 'money', 'fox  super monday': 'tv show/network', 'hyundai  smaht pahk': 'car', 'cheetos  can t touch this': 'food', 'olay  make space for women': 'charitable program', 'fox  halftime show  teaser_1': 'half-time show', 'michelob  6 for 6-pack': 'charitable program/ alcohol', 'avocados from mex

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
classification_ad_product = False

In [ ]:
product_id_dict = dict()
products = df['product_modified'].unique()
i=0

for product in products:
  product_id_dict[product] = i
  i = i+1

n_unique_ad_produts = len(product_id_dict)

if classification_ad_product:
  n_unique_ads = n_unique_ad_produts
  print(n_unique_ads)

df['ad_product_id'] = df['product_modified'].apply(lambda x: product_id_dict[x])

print("Size of dictionary:"+ str(len(product_id_dict)))

Size of dictionary:20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
# Remove ads with less than 2 samples

def getAdTweets(ad_related_twts, ad):
  return ad_related_twts[ad_related_twts.ad_manual_adjusted == ad].shape[0]

def get_filtered_data(df): 
  unique_ads = df['ad_manual_adjusted'].unique()
  #print(unique_ads)
  for ad in unique_ads : 
    if getAdTweets(df, ad) >=2:
        # do nothing
        print("")
    else:
      # print('ad with <2 samples: '+ str(ad))
      mask = df['ad_manual_adjusted'] != ad
      df = df[mask]
  return df

df = get_filtered_data(df)
print('Removed tweets for ads with <2 samples')






























































Removed tweets for ads with <2 samples


In [ ]:
# #Let us try some sampling technique to remove class imbalance
# from imblearn.over_sampling import SMOTE
# #Over-sampling: SMOTE
# #SMOTE (Synthetic Minority Oversampling TEchnique) consists of synthesizing elements for the minority class, 
# #based on those that already exist. It works randomly picking a point from the minority class and computing 
# #the k-nearest neighbors for this point.The synthetic points are added between the chosen point and its neighbors.
# #We'll use ratio='minority' to resample the minority class.
# smote = SMOTE('minority', k_neighbors = 3) # default value of n_neighbours = 6, but getting error since for a class n samples = 4 < default n_neighs which is 6

# input_data_raw = df['text_clean']
# input_labels_raw = df['ad_product_id']
# input_data, input_labels = smote.fit_sample(input_data_raw, input_labels_raw)
# print(input_data.shape, input_labels.shape)

In [ ]:
subset_ads = False

In [ ]:
def filter_ads(df):
  ads_needed = ['donald j. trump for president  criminal justice reform', 'hulu  tom brady s big announcement',
              'doritos  the cool ranch','olay  make space for women','planters  baby funeral',
              'amazon echo  before alexa', 'jeep  groundhog day [t1]'
              ,'none' # top 7
              # 'walmart  famous visitors',  'bud light seltzer  posty store  inside post s brain' 
              # ,'rocket mortgage  home', 'hyundai  smaht pahk', 'google assistant  loretta',
              # 'turbotax  turbotax  all people are tax people remix', 'jeep  groundhog day [t1]',
              # 'pringles  the infinite dimensions of rick and morty',
              # , 'audi  let it go [t1]' # top 15
              # ,'fox  halftime show  teaser_1', 'cheetos  can t touch this', 'fox  toads  the masked singer',
              # 'nfl  building a better game', 'snickers  fix the world', 't-mobile  mama tests 5g',
              # 'avocados from mexico  the avocados from mexico shopping network', 'budweiser  typical american',
              # 'coca-cola energy  show up', 'microsoft surface  be the one', 'sabra  how do you  mmus?',
              # 'verizon  the amazing things 5g won t do', 'genesis  going away party', 'mountain dew' #top 30
                ]
  print(len(ads_needed))
  mask = df['ad_manual_adjusted'].apply(lambda ad: ad in ads_needed)
  df_subset = df[mask]
  return df_subset

if subset_ads == True:
  df_subset = filter_ads(df)
  print(df_subset.shape)
  df_subset.head()
  df_all = df
  df = df_subset

In [ ]:
# df = df_all

In [ ]:
df[['ad_manual_adjusted', 'text_clean']].head()
count = df[['ad_manual_adjusted', 'text_clean']].groupby('ad_manual_adjusted').count()
count.head()
count.to_csv('./train_data_support.csv')

In [ ]:
# use column name 'ad_manual_adjusted' of df to 
def get_ad_related_twts(df, removeCommas = True):
  df['ad_manual_adjusted'] = df['ad_manual_adjusted'].apply(lambda x: x.lower())
  ad_filter = df['ad_manual_adjusted']!= 'none'
  ad_related_twts = df[ad_filter]
  if removeCommas:
    ad_filter_1 = ~ad_related_twts['ad_manual_adjusted'].str.contains(',')
    ad_related_twts = ad_related_twts[ad_filter_1]
  return ad_related_twts

def getAdTweets(ad_related_twts, ad):
  return ad_related_twts[ad_related_twts.ad_manual_adjusted == ad].shape[0]

def get_ad_id_dict(ad_related_twts): 
  n_ad_related = ad_related_twts.shape[0]
  print("# ad related tweets: "+ str(n_ad_related))
  ads_annotated = ad_related_twts.ad_manual_adjusted.values
  adset = set(ads_annotated)
  print("unique ads:"+ str(len(adset)))
  ad_id_dict = {}
  i = 0
  for ad in adset : 
    if(getAdTweets(ad_related_twts, ad) >=2):
      ad_id_dict[ad] = i
      i = i+1
    else:
      print('ad with <2 samples: '+ str(ad))
  print(" No of ads with >=2 samples:"+ str(len(ad_id_dict)))
  ad_id_dict['none'] = len(ad_id_dict)
  print(ad_id_dict)
  return ad_id_dict

def convertAdNameToAdId(ad_id_dict, ad_name):
  if ad_name in ad_id_dict:
    return ad_id_dict[ad_name]
  else:
    return ad_id_dict['none']

ad_related_twts = get_ad_related_twts(df)
ad_id_dict = get_ad_id_dict(get_ad_related_twts(df))
n_unique_ads = len(ad_id_dict) # ad_id_dict has none as well, so minus 1 when using embeddings
df['ad_manual_adjusted_id'] = df['ad_manual_adjusted'].apply(lambda x: convertAdNameToAdId(ad_id_dict,x))
print(n_unique_ads)

# ad related tweets: 2607
unique ads:60
 No of ads with >=2 samples:60
{'fox  toads  the masked singer': 0, 'michelob  jimmy works it out': 1, 'hyundai  smaht pahk': 2, 'jeep  groundhog day [t1]': 3, 'heinz ketchup  find the goodness  four at once': 4, 'hummer  gmc  quiet revolution': 5, 'google assistant  loretta': 6, 'genesis  going away party': 7, 'marvel  black widow trailer': 8, 'turbotax  turbotax  all people are tax people remix': 9, 'reese s  rock': 10, 'microsoft surface  be the one': 11, 'rocket mortgage  home': 12, 'fox  a run at history  daytona 500': 13, 'sabra  how do you  mmus?': 14, 'nfl  inspire change  anquan boldin': 15, 'fox  chosen  lego masters': 16, 'budweiser  typical american': 17, 'little caesars pizza  best thing since sliced bread': 18, 'facebook  ready to rock?': 19, 'squarespace  winona in winona': 20, 'fox  great american race': 21, 'snickers  fix the world': 22, 'amazon prime video  hunters': 23, 'weathertech  lucky dog': 24, 'amazon echo  before alexa':

In [ ]:
classification_ad_product = False # enable this flag if labels are ad products(i.e buckets of ad names) instead of ad names

In [ ]:
if classification_ad_product:
  twt_train, twt_test, label_train, label_test = train_test_split(df['text_clean'], 
                                                      df['product_modified'], test_size=0.2, random_state=42
                                                      , stratify= df.ad_manual_adjusted.values)
  unique_ads = df['product_modified'].unique()
  print("No of Unique ads:"+ str(len(unique_ads)))
else:
  twt_train, twt_test, label_train, label_test = train_test_split(df['text_clean'], 
                                                      df['ad_manual_adjusted'], test_size=0.2, random_state=42
                                                      , stratify= df.ad_manual_adjusted_id.values)
  unique_ads = df['ad_manual_adjusted'].unique()
  print("No of Unique ads:"+ str(len(unique_ads)))

No of Unique ads:61


#### Exploding sentences

In [ ]:
sentence_explode = True
sentences, test_sentences, labels, test_labels = twt_train, twt_test, label_train, label_test

In [ ]:
# group the keywords of ads part of a product bucket and append these to the training sentences
ad_product_keywords_dict = ad_product_df.groupby('Product_modified')['Key Terms  Round 2'].agg(lambda x : x.sum() if x.dtype=='float64' else ' '.join(x))
# clean the ad key words - not removing duplicate words here - TODO
for ad_bucket in ad_product_keywords_dict.keys():
  ad_product_keywords_dict[ad_bucket] = cleanTweet(ad_product_keywords_dict[ad_bucket])
ad_product_df['product_modified_keywords'] = ad_product_df['Product_modified'].apply(lambda x: ad_product_keywords_dict[x])
ad_product_df.head(2)

# generate adname, ad keywords dict to use in sent exploding
ad_name_keywords_dict = dict()
for i, row in ad_product_df.iterrows():
  ad_name_keywords_dict[row['Ad_Name'].lower()] = cleanTweet(row['Key Terms  Round 2'])
ad_name_keywords_dict['none'] = 'none'
print(ad_name_keywords_dict)

ad_keywords_name_dict = dict()
for ad_name in ad_name_keywords_dict:
  keywords_temp = ad_name_keywords_dict[ad_name]
  ad_keywords_name_dict[keywords_temp] = ad_name
print(ad_keywords_name_dict)

ad_product_df['ad_name_keywords'] = ad_product_df['Ad_Name'].apply(lambda x: ad_name_keywords_dict[x.lower()])

{'fast & furious 9  trailer': 'fast and the furious fast the furious ff f the fast saga vin diesel flying truck stunts michelle rodriguez fastfurious', 'quibi  bank heist': 'quibi bank heist robbery less than ten minutes quick bites big stories chance the rapper megan thee stallion chancetherappermeganstallion quickbites', 'tide  when is later  masked singer': 'tide laundry laundry detergent schitt emily hampshire charlie day walts', 'fox  a run at history  daytona 500': 'fox show daytona a run at history great american race', 'donald j. trump for president  criminal justice reform': 'donald trump trump change unemployment stronger safer prosperous trumpforpresident donaldtrump presidentrumpobama gop trumpsupporters', 'walmart  famous visitors': 'walmart pickup spaceship toy story buzz lightyear marvin the martian marvin martians arrival glass cleaners aliens men in black groot lego star wars r d bill bill and ted flash gordon', 'marvel  black widow trailer': 'marvel black widow scarle

In [ ]:
# explode to only 0,1 classes
def explode_data_to_two_classes(sentences, labels, classification_ad_product):
  label_name_list = []
  label_id_list = []
  if classification_ad_product:
    label_name_list = list(product_id_dict.keys())
    label_id_list = list(product_id_dict.values())
  else:
    label_name_list = list(ad_id_dict.keys())
    label_id_list = list(ad_id_dict.values())

  sentences_exploded =[]
  labels_exploded = []
  # sent_labels_df = pd.DataFrame()
  for i in range(len(sentences)):
    curr_sent = sentences[i]
    curr_label_name = labels[i]

    curr_sent_exploded = []
    curr_label_exploded = []
    for j in range(len(label_name_list)): # expand for all products present
      if label_name_list[j] is not 'none':
        if classification_ad_product:
          label_keywords = ad_product_keywords_dict[label_name_list[j]]
        else:
          label_keywords = ad_name_keywords_dict[label_name_list[j]]

        curr_sent_exploded.append(curr_sent+". "+ label_keywords)
          
        if label_name_list[j] == curr_label_name :
          curr_label_exploded.append(1)
        else:
          curr_label_exploded.append(0)
    sentences_exploded.extend(curr_sent_exploded)
    labels_exploded.extend(curr_label_exploded)

  return sentences_exploded, labels_exploded

In [ ]:
if sentence_explode:
  sentences_exploded, labels_exploded = explode_data_to_two_classes(sentences.values, labels.values, classification_ad_product)
  print("size of train data after exploding: "+ str(len(sentences_exploded)))
  test_sentences_exploded, test_labels_exploded = explode_data_to_two_classes(test_sentences.values, test_labels.values, classification_ad_product)
  print("size of test data after exploding: "+ str(len(test_sentences_exploded)))

size of train data after exploding: 280380
size of test data after exploding: 70140


#### Methods to extract and process Results

In [ ]:
def get_sent_exploded_results(pred_scores, pred_labels, opFileName):

  sent_exploded_results_df = pd.DataFrame()
  sent_exploded_results_df['sentence_raw'] = test_sentences_exploded
  sent_exploded_results_df['label_pred'] = pred_labels
  sent_exploded_results_df['label_true'] = test_labels_exploded
  sent_exploded_results_df['pred_scores'] = pred_scores
  sent_exploded_results_df[['sentence','keywords_appended']] = sent_exploded_results_df.sentence_raw.str.split(".",expand=True) 

  sent_exploded_results_df['keywords_appended_name'] = sent_exploded_results_df['keywords_appended'].apply(lambda x: 'none' if x is None else ad_keywords_name_dict[x.strip()])

  sent_exploded_results_df.to_csv("./"+opFileName)
  print(sent_exploded_results_df.head())
  return sent_exploded_results_df


In [ ]:
def get_pred_2_label_name_score(df):
  sentence = df['sentence'].iloc[0:1] # since grouped by sentence there will be only one unique sentence
  max_pred_score = 0
  pred_label = 0
  pred_label_name = "none"

  true_label = 0
  true_label_name = "none"

  for i, row in df.iterrows():
    if row['label_pred'] == 1 and row['pred_scores']> max_pred_score:
      max_pred_score = row['pred_scores']
      pred_label = 1
      pred_label_name = row['keywords_appended_name']
    # if all 0  => not related to any ad => none
    # if all 2 => none

    # check for the true label as well 
    if row['label_true'] == 1 :
        true_label = 1
        true_label_name = row['keywords_appended_name']
   # default 0 which will be none     

  result = pd.DataFrame()
  result['sentence'] = sentence
  result['pred_label_name'] = pred_label_name
  result['max_pred_score'] = max_pred_score
  result['true_label'] = true_label
  result['true_label_name'] = true_label_name
  return result

In [ ]:
from sklearn.metrics import classification_report
def getClassificationReport(sent_exploded_results_df, opFileName1, opFileName2):
  sentence_results_df = pd.DataFrame()
  sentence_results_df[['sentence_1', 'pred_label_name', 'max_pred_score', 'true_label', 'true_label_name']] = sent_exploded_results_df.groupby('sentence').apply(get_pred_2_label_name_score)

  print(sentence_results_df.shape)
  print(sentence_results_df.head())
  sentence_results_df.to_csv('./'+ opFileName1)

  true_names = sentence_results_df['true_label_name']
  pred_names = sentence_results_df['pred_label_name']
  class_report = classification_report(true_names, pred_names,output_dict=True)

  classification_report_df = pd.DataFrame(class_report).transpose()
  print(classification_report_df.head())
  classification_report_df.to_csv('./'+ opFileName2)
  return classification_report_df

ANN

In [ ]:
# Run this to free up RAM
# del(X_train)
# del(X_test)

In [ ]:
# first neural network with keras make predictions
from keras.models import Sequential
from keras.layers import Dense
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow
from sklearn.preprocessing import LabelEncoder

if sentence_explode:
  print("Joint Learning")
  sentences = sentences_exploded
  test_sentences = test_sentences_exploded
  labels = labels_exploded
  label_test = test_labels_exploded

# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer(max_features = 6000) #max_features = 150
vectorizer.fit(sentences)
X_train = vectorizer.transform(sentences)
X_test  = vectorizer.transform(test_sentences)
input_dim = X_train.shape[1] 


Joint Learning


In [ ]:
X_train.shape

(280380, 6100)

In [ ]:
# fit the keras model on the dataset
if sentence_explode:
  no_classes = 2
else:
  no_classes = 61
X_train = X_train.toarray()
X_test = X_test.toarray()


In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim= input_dim, activation='relu'))
# model.add(Dense(8, activation='relu'))
if sentence_explode:
  model.add(Dense(no_classes, activation='softmax')) #sigmoid
else:
  model.add(Dense(no_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)# label_train?
encoded_train = encoder.transform(labels)
one_hot_y = tensorflow.keras.utils.to_categorical(encoded_train)

print(encoded_train.shape)
print(one_hot_y.shape)
print(X_train.shape)

model.fit(X_train, one_hot_y , epochs=50, batch_size=200)

encoded_test = encoder.transform(label_test)

# make class predictions with the model
label_pred_ann = model.predict_classes(X_test) ##   label_pred = np.argmax(pred_probs, axis=1)
pred_probs = model.predict_proba(X_test)
label_pred_ann_scores = np.max(pred_probs, axis=1)

(280380,)
(280380, 2)
(280380, 6100)
Epoch 1/50
1402/1402 [==============================] - 26s 8ms/step - loss: 0.2283 - accuracy: 0.9797
Epoch 2/50
1402/1402 [==============================] - 11s 8ms/step - loss: 0.0389 - accuracy: 0.9923
Epoch 3/50
1402/1402 [==============================] - 11s 8ms/step - loss: 0.0355 - accuracy: 0.9926
Epoch 4/50
1402/1402 [==============================] - 11s 8ms/step - loss: 0.0346 - accuracy: 0.9926
Epoch 5/50
1402/1402 [==============================] - 10s 7ms/step - loss: 0.0340 - accuracy: 0.9927
Epoch 6/50
1402/1402 [==============================] - 11s 8ms/step - loss: 0.0345 - accuracy: 0.9925
Epoch 7/50
1402/1402 [==============================] - 11s 8ms/step - loss: 0.0322 - accuracy: 0.9927
Epoch 8/50
1402/1402 [==============================] - 11s 8ms/step - loss: 0.0334 - accuracy: 0.9924
Epoch 9/50
1402/1402 [==============================] - 11s 8ms/step - loss: 0.0338 - accuracy: 0.9923
Epoch 10/50
1402/1402 [=============

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
sent_exploded_results_df_ann = get_sent_exploded_results(label_pred_ann_scores,label_pred_ann,\
                                                              "sent_exploded_results_df_ann.csv" )
class_report_ann = getClassificationReport(sent_exploded_results_df_ann, "sent_exp_group_results_ann.csv",\
                        "classification_report_sent_explode_ann.csv")

In [ ]:
stop here for ann

In [ ]:
# for multi class classification ot sent_explode = False
classification_report_ann = classification_report(encoded_test, label_pred, target_names= set(label_test),output_dict=True)
classification_report_ann_df = pd.DataFrame(classification_report_ann).transpose()
if sentence_explode: 
  classification_report_ann_df.to_csv('./classification_report_ann_exploded.csv')
else:
  classification_report_ann_df.to_csv('./classification_report_ann.csv')

In [ ]:
# multi-class classification with Keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

input_dim = X_train.shape[1] 
n_classes = 2
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=input_dim, activation='relu'))
	model.add(Dense(n_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=100, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X_train.astype(float), one_hot_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
stop here

#### Naive Bayes Classifier for Multinomial Models

In [ ]:

# nb = Pipeline([('vect', CountVectorizer()),
#                ('tfidf', TfidfTransformer()),
#                ('clf', MultinomialNB()),
#               ])
# nb.fit(twt_train, label_train)

# label_pred_nb = nb.predict(twt_test)


# print('accuracy %s' % accuracy_score(label_pred_nb, label_test))
# classification_report_nb = classification_report(label_test, label_pred_nb, target_names= set(label_test),output_dict=True)
# classification_report_nb_df = pd.DataFrame(classification_report_nb).transpose()
# classification_report_nb_df.to_csv('./classification_report_nb.csv')


#### Linear Support Vector Machine


In [ ]:
# from sklearn.linear_model import SGDClassifier

# sgd = Pipeline([('vect', CountVectorizer()),
#                 ('tfidf', TfidfTransformer()),
#                 ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=10, tol=None)),
#                ])
# if sentence_explode:
#   sgd.fit(sentences_exploded, labels_exploded)
  
# else:
#   sgd.fit(twt_train, label_train)
#   label_pred_sgd = sgd.predict(twt_test)

#   print('accuracy %s' % accuracy_score(label_pred_sgd, label_test))
#   classification_report_sgd = classification_report(label_test, label_pred_sgd, target_names= set(label_test), output_dict=True)
#   classification_report_sgd_df = pd.DataFrame(classification_report_sgd).transpose()
#   classification_report_sgd_df.to_csv('./classification_report_sgd.csv')

In [ ]:
# from sklearn.linear_model import SGDClassifier
# from sklearn.calibration import CalibratedClassifierCV
# sentence_explode = True
# if sentence_explode:
#   weights = {0:1, 1:1000}
#   sgd = Pipeline([('vect', CountVectorizer()),
#                 ('tfidf', TfidfTransformer()),
#                 ('clf', CalibratedClassifierCV((SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3,\
#                         random_state=42, max_iter=5, tol=None, class_weight=weights)))),
#                ])
#   sgd.fit(sentences_exploded, labels_exploded)
#   pred_probs = sgd.predict_proba(test_sentences_exploded)
#   label_pred_sgd = np.argmax(pred_probs, axis=1)
#   label_pred_sgd_scores = np.max(pred_probs, axis=1)
#   classes = sgd.classes_
#   print(label_pred_sgd_scores.shape)
#   # label_pred_sgd = sgd.predict(test_sentences_exploded)
#   print('accuracy %s' % accuracy_score(label_pred_sgd, test_labels_exploded)) #accuracy 0.9898916452808668
#   sent_exploded_results_df_sgd = get_sent_exploded_results(label_pred_sgd_scores,label_pred_sgd,\
#                                                               "sent_exploded_results_df_sgd.csv" )
#   class_report_sgd = getClassificationReport(sent_exploded_results_df_sgd, "sent_exp_group_results_sgd.csv",\
#                         "classification_report_sent_explode_sgd.csv")
# else:
#   sgd.fit(twt_train, label_train)
#   label_pred_sgd = sgd.predict(twt_test)
#   print('accuracy %s' % accuracy_score(label_pred_sgd, label_test))
#   classification_report_sgd = classification_report(label_test, label_pred_sgd, target_names= set(label_test), output_dict=True)
#   classification_report_sgd_df = pd.DataFrame(classification_report_sgd).transpose()
#   classification_report_sgd_df.to_csv('./classification_report_sgd.csv')
  

#### Non Linear SVM with TF-IDF vectorization. 

In [ ]:
##SVM
from sklearn import svm, datasets
from sklearn.multiclass import OneVsOneClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

weights = {0:1, 1:100} # SVM with weights: https://machinelearningmastery.com/cost-sensitive-svm-for-imbalanced-classification/

model = svm.SVC(gamma = 'scale', probability=True)

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', model)
               ])
data_size = len(sentences_exploded)
chunk_size = 10000
n_iters = 1+ int(data_size/chunk_size)
print(n_iters)
for i in range(0, n_iters):
  startInd = i*10000
  endInd = min((i+1)*chunk_size, data_size)
  sgd.fit(sentences_exploded[ startInd: endInd], labels_exploded[startInd: endInd])
  print(i)

pred_probs = sgd.predict_proba(test_sentences_exploded)
label_pred_sgd = np.argmax(pred_probs, axis=1)
label_pred_sgd_scores = np.max(pred_probs, axis=1)
classes = sgd.classes_
print(label_pred_sgd_scores.shape)
print('accuracy %s' % accuracy_score(label_pred_sgd, test_labels_exploded)) #accuracy 0.9898916452808668
sent_exploded_results_df_sgd = get_sent_exploded_results(label_pred_sgd_scores,label_pred_sgd,\
                                                              "sent_exploded_results_df_sgd.csv" )
class_report_sgd = getClassificationReport(sent_exploded_results_df_sgd, "sent_exp_group_results_sgd.csv",\
                        "classification_report_sent_explode_sgd.csv")

In [ ]:
class_report_sgd.head(10)

In [ ]:
stop here

#### SVM with TF-IDF vectorization and One Vs One classifier 

**Issues:**
 

1.   One Vs One not specifically needed since in Joint Learning there are only two classes
2.   one Vs one does not support predict_proba


# New Section

In [ ]:
# works only to predict classes but not probabilities hence classification
# within ads is not possible in joint learning setting
# ref one-vs-one classifier: https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/
from sklearn import svm, datasets
from sklearn.multiclass import OneVsOneClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV

model = svm.SVC(gamma = 'scale', probability=True)
ovo = OneVsOneClassifier(model)

# tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
tfidf = TfidfVectorizer(max_features = 500)
sentences_exploded_features = tfidf.fit_transform(sentences_exploded)
print(sentences_exploded_features.shape)

data_size = len(sentences_exploded)
chunk_size = 5000
n_iters = 1+ int(data_size/chunk_size)
print(n_iters)
for i in range(0, n_iters):
  startInd = i*chunk_size
  endInd = min((i+1)*chunk_size, data_size)
  ovo.fit(sentences_exploded_features[startInd: endInd], labels_exploded[startInd: endInd])
  print(i)


one Vs One classifier cannot predict probabilities hence this code only gives results from 0 Vs 1 classes in joint learning setting


In [ ]:
test_sentences_exploded_features = tfidf.fit_transform(test_sentences_exploded)
label_pred_sgd_scores =  ovo.predict(test_sentences_exploded_features)
print('accuracy %s' % accuracy_score(label_pred_sgd_scores, test_labels_exploded))
classification_report_sgd = classification_report(test_labels_exploded, label_pred_sgd_scores, target_names= set(test_labels_exploded), output_dict = True)
classification_report_sgd_df = pd.DataFrame(classification_report_sgd).transpose()
classification_report_sgd_df.to_csv('./classification_report_sgd.csv')

# one Vs One classifier cannot predict probabilities hence this code does not work 

# test_sentences_exploded_features = tfidf.fit_transform(test_sentences_exploded)
# pred_probs = ovo.predict_proba(test_sentences_exploded_features)
# label_pred_sgd = np.argmax(pred_probs, axis=1)
# label_pred_sgd_scores = np.max(pred_probs, axis=1)
# classes = ovo.classes_
# print(label_pred_sgd_scores.shape)
# print('accuracy %s' % accuracy_score(label_pred_sgd, test_labels_exploded)) #accuracy 0.9898916452808668
# sent_exploded_results_df_sgd = get_sent_exploded_results(label_pred_sgd_scores,label_pred_sgd,\
#                                                               "sent_exploded_results_df_sgd.csv" )
# class_report_sgd = getClassificationReport(sent_exploded_results_df_sgd, "sent_exp_group_results_sgd.csv",\
#                         "classification_report_sent_explode_sgd.csv")
# ovo.partial_fit(sentences_exploded_features, labels_exploded, classes = list(set(labels_exploded)))

In [ ]:
classification_report_sgd_df.head(15)

#### LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5, solver = 'saga', max_iter = 5000)),
               ])
if sentence_explode:
  logreg.fit(sentences_exploded, labels_exploded)
  pred_probs = logreg.predict_proba(test_sentences_exploded)
  label_pred_logreg = np.argmax(pred_probs, axis=1)
  label_pred_logreg_scores = np.max(pred_probs, axis=1)
  classes = logreg.classes_
  label_pred_logreg = [classes[i] for i in label_pred_logreg]
  print('accuracy %s' % accuracy_score(label_pred_logreg, test_labels_exploded)) #accuracy 0.857

else:
  logreg.fit(twt_train, label_train)
  label_pred_logreg = logreg.predict(twt_test)
  print('accuracy %s' % accuracy_score(label_pred_logreg, label_test)) #accuracy 0.8306244653550042
  classification_report_logreg = classification_report(label_test, label_pred_logreg, target_names= set(label_test), output_dict=True)
  classification_report_logreg_df = pd.DataFrame(classification_report_logreg).transpose()
  classification_report_logreg_df.to_csv('./classification_report_logreg.csv')



In [ ]:
sent_exploded_results_df_log_reg = get_sent_exploded_results(label_pred_logreg_scores,label_pred_logreg,\
                                                              "sent_exploded_results_df_logreg.csv" )
getClassificationReport(sent_exploded_results_df_log_reg, "sent_exp_group_results_logReg.csv",\
                        "classification_report_sent_explode_logReg.csv")

#### Plots

In [ ]:
train_data = pd.DataFrame()
train_data['data'] = twt_train
train_data['label'] = label_train
train_data.to_csv('./log_reg_train_data.csv')

In [ ]:
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    # print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

y_pred = label_pred_logreg
cnf_matrix = confusion_matrix(label_test, y_pred)

# Plot normalized confusion matrix
fig = plt.figure()
fig.set_size_inches(50, 50, forward=True)
#fig.align_labels()

# fig.subplots_adjust(left=0.0, right=1.0, bottom=0.0, top=1.0)
plot_confusion_matrix(cnf_matrix, classes=np.asarray(unique_ads), normalize=True,
                      title='Normalized confusion matrix')